#  Coursera Capstone
This notebook will be used for the IBM's Applied Data Science Capstone Project.<br>
Weekly assignments will be implemented on the same notebook.

### Week 1 <br>
Importing the necessary libraries


In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Week 2<br>
Scraping the wikipedia webpage to retrieve the table with the data

In [150]:
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
Postal_Code=[]
Borough=[]
Neighbourhood=[]
for items in soup.find('table', class_='wikitable').find_all('tr'):
    data = items.find_all(['th','td'])
    try:
        Postal_Code.append(data[0].text.replace('\n',''))
        Borough.append(data[1].text.replace('\n',''))
        Neighbourhood.append(data[1].find_next_sibling().text.replace('\n',''))
    except IndexError:pass
#print("{}|{}|{}".format(Postal_Code,Borough,Neighbourhood))

Creating the dataframe with the three columns and <br>
making the first table row index

In [151]:
df=pd.DataFrame({'Postal_Code':Postal_Code,'Borough':Borough,'Neighbourhood':Neighbourhood})
new_header = df.iloc[0] 
df = df[1:] 
df.columns = new_header
df.head(10)

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
10,M1B,Scarborough,"Malvern, Rouge"


Drop rows with 'Not assigned' Borough value

In [152]:
df['Borough'].replace('Not assigned',np.NaN,inplace=True)
df.dropna(axis=0,inplace=True)
df.reset_index( drop=True,inplace=True)
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Grupping Neighbourhoods by Borough

In [153]:
df.groupby('Postal Code').agg(lambda x: x.tolist())
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Replacing all 'Not assigned' Neighbourhoods with Borough values

In [154]:
df['Neighbourhood'][df['Neighbourhood']=='Not assigned']=df['Borough']
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [155]:
df.shape

(103, 3)

<b>Getting the coordinates for Toronto

In [165]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_dc634226746e43d0bacc8ef81627305b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fIrt3A0GR7UBCSqU5DPpbiIHX7jiT9y_t7gH9T1BHtwz',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_dc634226746e43d0bacc8ef81627305b.get_object(Bucket='courseracapstoneproject-donotdelete-pr-zspibbsqs5lgj8',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two data frames with respect o the key value 'Posta Code'

In [173]:
df=pd.merge(df, df_data_1, on="Postal Code")
df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,43.753259,-79.329656,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,43.657162,-79.378937


In [178]:
from project_lib import Project
sc
project = Project(,"095590fd-239e-4fe2-bbd3-890faae431","p-8a4770edd954f9cd0ac21bf9211c88534a7ef338")
project.save_data(file_name = "mypanda.csv",data = df.to_csv(index=False))

2021-01-20 12:53:54,783 - __PROJECT_LIB__ - ERROR - GET project failed, status 400 from https://api.eu-gb.dataplatform.cloud.ibm.com/v2/projects/p-8a4770edd954f9cd0ac21bf9211c88534a7ef338
{"code":400,"error":"Bad Request","reason":"Invalid project GUID encountered in request path: p-8a4770edd954f9cd0ac21bf9211c88534a7ef338.","message":"The server cannot or will not process the request due to an apparent client error (e.g. malformed request syntax)."}


RuntimeError: Failed to GET project, status: 400